We aim to predict travel time minutes per trip id using features like route id , hour of the dat and route type 

In [1]:
#libraries
import pandas as pd
import os
print(os.getcwd())

C:\Users\S P E C T R E\Desktop\Personal Projects\umpire


In [2]:
base_path = r"C:\Users\S P E C T R E\Desktop\Personal Projects\umpire\gtfs-accra-ghana-2016"

In [3]:
stop_times = pd.read_csv(f"{base_path}\\stop_times.csv")
trips = pd.read_csv(f"{base_path}\\trips.csv")
routes = pd.read_csv(f"{base_path}\\routes.csv")
calendar = pd.read_csv(f"{base_path}\\calendar.csv")
shapes = pd.read_csv(f"{base_path}\\shapes.csv")
stops = pd.read_csv(f"{base_path}\\stops.csv")
fare_rules = pd.read_csv(f"{base_path}\\fare_rules.csv")
fare_attributes = pd.read_csv(f"{base_path}\\fare_attributes.csv")
agency = pd.read_csv(f"{base_path}\\agency.csv")

### Calculating travel time per trip

In [4]:
#convert times to timedelta
stop_times["arrival_time"] = pd.to_timedelta(stop_times["arrival_time"], errors = "coerce")
stop_times["departure_time"] = pd.to_timedelta(stop_times["departure_time"], errors = "coerce")

#trip start and end times
trip_durations = (
    stop_times.groupby('trip_id')
    .agg(start_time=('departure_time', 'min'), end_time=('arrival_time', 'max'))
    .reset_index()
)

# Calculate travel time in minutes
trip_durations['travel_time_minutes'] = (trip_durations['end_time'] - trip_durations['start_time']).dt.total_seconds() / 60
trip_durations.head()

,trip_id,start_time,end_time,travel_time_minutes
0,002A_1,0 days 12:51:38,0 days 13:10:58,19.333333
1,002B_1,0 days 13:16:27,0 days 13:44:40,28.216667
2,003A_1,0 days 13:51:37,0 days 14:45:08,53.516667
3,003B_1,0 days 15:28:50,0 days 15:55:50,27.000000
4,004A_1,0 days 13:22:52,0 days 14:09:22,46.500000


### Feature Engineering - Route info + hour of day

In [5]:
print(trips.columns.tolist())

['route_id', 'service_id', 'trip_id', 'shape_id']


In [6]:
# Merge trip_durations with trips.csv
trip_durations = trip_durations.merge(trips[['trip_id', 'route_id', 'service_id']], on='trip_id', how='left')

# Merge with routes.csv to get route_type
trip_durations = trip_durations.merge(routes[['route_id', 'route_type']], on='route_id', how='left')

# Get the first stop (where trip starts)
first_stops = stop_times.groupby('trip_id').first().reset_index()

# Extract hour from departure time
first_stops['hour_of_day'] = first_stops['departure_time'].dt.components['hours']  #this feature is to represent what time of day a trip starts. helps with rush hour  

# Merge hour_of_day into the trip_durations DataFrame
df = trip_durations.merge(first_stops[['trip_id', 'hour_of_day']], on='trip_id', how='left')
df.head()

,trip_id,start_time,end_time,travel_time_minutes,route_id,service_id,route_type,hour_of_day
0,002A_1,0 days 12:51:38,0 days 13:10:58,19.333333,002A,service,3,12
1,002B_1,0 days 13:16:27,0 days 13:44:40,28.216667,002B,service,3,13
2,003A_1,0 days 13:51:37,0 days 14:45:08,53.516667,003A,service,3,13
3,003B_1,0 days 15:28:50,0 days 15:55:50,27.000000,003B,service,3,15
4,004A_1,0 days 13:22:52,0 days 14:09:22,46.500000,004A,service,3,13


In [8]:
df.shape  # returns (num_rows, num_columns)

(651, 8)

In [9]:
df.isnull().sum()

trip_id                0
start_time             0
end_time               0
travel_time_minutes    0
route_id               0
service_id             0
route_type             0
hour_of_day            0
dtype: int64